<a href="https://colab.research.google.com/github/gayeoniee/data_anal_DL/blob/main/1_%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%98_%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# MLPClassifier로 신경망 구성

In [ ]:
from sklearn.neural_network import MLPClassifier
# MLP(다층퍼셉트론) 분류기 만들기: 은닉층 50 -> 30, 활성화 logistic(=시그모이드), 옵티마이저 adam
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), activation='logistic', solver='adam', max_iter=1000)

In [ ]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [ ]:
mlp.score(X_test, y_test)

1.0

In [ ]:
pred = mlp.predict(X_test)

import pandas as pd
pd.crosstab(pred, y_test)  # 교차분류표

col_0,0,1,2
row_0,,,
0,19,0,0
1,0,13,0
2,0,0,13


**시그모이드(Sigmoid)**   

어떤 입력값이 오든 0~1 사이로 바꿔주는 함수   
입력이 크면 1에 가까워지고 작으면 0에 가까워짐. 0이면 0.5


**소프트맥스(Softmax)**

여러 개의 점수(로짓)를 확률처럼 바꿔서 모든 값 합=1 이 되게 만드는 함수   
다중분류(서로 배타적인 3개 이상 클래스) 의 마지막 층에서 각 클래스의 확률이 필요할 때 사용

# 텐서플로 신경망 구성

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
X = tf.placeholder(tf.float32, [None, 4])  # 입력 자리(배치크기 가변, 특징 4개)
y = tf.placeholder(tf.float32, [None, 3])  # 정답 자리(3개 클래스)

## W 가중치, b 편향

In [ ]:
w1 = tf.Variable(tf.random_normal([4, 50]))  # 가중치(입력4, 은닉50)
b1 = tf.Variable(tf.zeros([50]))             # 편향(은닉50개에 각각 1개)
h1 = tf.nn.sigmoid(tf.matmul(X, w1) + b1)    # 선형결합 + 시그모이드

## 2layer - 30node 로 구성

In [ ]:
w2 = tf.Variable(tf.random_normal([50, 30]))  # 가중치(은닉50, 은닉30)
b2 = tf.Variable(tf.zeros([30]))              # 편향(30개)
h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)    # 은닉1 출력 -> 은닉2로

## 3layer 출력층

In [ ]:
w3 = tf.Variable(tf.random_normal([30, 3]))   # 가중치(은닉30, 클래스3)
b3 = tf.Variable(tf.zeros([3]))               # 편향(3개)
h3 = tf.nn.softmax(tf.matmul(h2, w3) + b3)    # 각 클래스 확률(합=1)

In [ ]:
# 손실: 교차엔트로피(직접 소프트맥스 후 log를 쓰는 단순식)
cross_entropy = -tf.reduce_sum(y * tf.log(h3))       # 예측확률이 정답에 가까울수록 작아짐
loss = tf.reduce_mean(cross_entropy)                 # 배치 평균 손실
train = tf.train.AdamOptimizer(0.01).minimize(loss)  # Adam으로 손실 줄이기(학습)

In [ ]:
# 피드(입력값 넣기) 준비: y를 one-hot로 변환
feed_train = {X: X_train, y: pd.get_dummies(y_train).values}

init = tf.global_variables_initializer()   # 변수 초기화(W,b 첫값 세팅)
sess = tf.Session()    # 그래프 실행용 세션 만들기
sess.run(init)

for i in range(1000):      # 1000번 학습 반복(에폭 느낌)
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss))

Step 0, 355.46481
Step 100, 12.09117
Step 200, 6.03688
Step 300, 5.49801
Step 400, 5.20966
Step 500, 4.93799
Step 600, 4.57616
Step 700, 3.53050
Step 800, 2.11480
Step 900, 1.14922


In [ ]:
# 정확도 계산: 예측 라벨(argmax)와 실제 라벨(one-hot argmax) 비교
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {X: X_test, y: pd.get_dummies(y_test).values}     # 테스트 피드
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [ ]:
sess.close()    # 세션 닫기(자원 정리)

# keras로 신경망 구성

세션 재시작 후 실행

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()       # 층을 차곡차곡 쌓는 그릇
model

<Sequential name=sequential, built=False>

In [ ]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(shape=(4,)))                      # 입력 크기 4(특징 4개)
model.add(Dense(50, activation='sigmoid'))        # 은닉층 50, 시그모이드
model.add(Dense(30, activation='sigmoid'))        # 은닉층 30, 시그모이드
model.add(Dense(3, activation='softmax'))         # 출력층 3, 소프트맥스 확률

In [ ]:
model.compile(
    optimizer = 'adam',                          # 파라미터 업데이트 방법
    loss = 'sparse_categorical_crossentropy',    # 라벨이 정수(0/1/2)일 때 쓰는 손실
    metrics = ['accuracy']
)

In [ ]:
model

<Sequential name=sequential, built=True>

In [ ]:
model.fit(X_train, y_train, epochs=300, verbose=True)

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3149 - loss: 1.4475
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3628 - loss: 1.3442 
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3347 - loss: 1.3031 
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3618 - loss: 1.2028
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3420 - loss: 1.1785
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3472 - loss: 1.1326
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3326 - loss: 1.1076
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3357 - loss: 1.0876 
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3639 - loss: 1.0666
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3291 - loss: 1.0656
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6015 - loss: 1.0513
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4031 - 

In [ ]:
model.evaluate(X_test, y_test)       # 테스트셋 성능(손실, 정확도) 계산

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0597 


[0.05596361681818962, 1.0]

# PyTorch로 구성하기

세션 재시작 후 실행

In [ ]:
import torch
torch.__version__

'2.8.0+cu126'

In [ ]:
if torch.cuda.is_available():        # GPU가 있다면 이름 표시(없으면 생략)
  torch.cuda.get_device_name(0)

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 넘파이 → 파이토치 텐서로 변환 (float는 실수형)
X_train = torch.tensor(X_train, dtype=torch.float32)    # [N,4]
y_train = torch.tensor(y_train, dtype=torch.float32)    # [N] (나중에 long으로 변환)
X_test = torch.tensor(X_test, dtype=torch.float32)      # [M,4]
y_test = torch.tensor(y_test, dtype=torch.float32)      # [M]

In [ ]:
import torch.nn as nn
model = nn.Sequential(
            nn.Linear(4, 50), nn.Sigmoid(),       # 4→50, 시그모이드
            nn.Linear(50, 30), nn.Sigmoid(),      # 50→30, 시그모이드
            nn.Linear(30, 3), nn.Softmax(dim=1),  # 30→3, 소프트맥스(확률)
        )

In [ ]:
# W의 초기값 설정
for layer in model:
  if isinstance(layer, nn.Linear):
    nn.init.xavier_uniform_(layer.weight)  # 가중치 초기값을 Xavier(Glorot) 방식으로 세팅(학습 잘 되게 도와줌)

# 손실 함수: 다중분류용
crit = nn.CrossEntropyLoss()     # 내부적으로 LogSoftmax+NLLLoss 조합

# 옵티마이저
import torch.optim as optim
opt = optim.Adam(model.parameters())

In [ ]:
# 학습 설정
for epoch in range(300):
  outputs = model(X_train)         # 순전파(logits 또는 확률)
  loss = crit(outputs, y_train.long())    # 손실값
  opt.zero_grad()  # 새로운 W 계산  -> 이전 기울기(gradient) 초기화
  loss.backward()         # 역전파(gradient 계산)
  opt.step()        # 가중치 업데이트

In [ ]:
# 모델 평가
with torch.no_grad():
  test_outputs = model(X_test)    # 테스트셋 예측
  _, predicted = torch.max(test_outputs, 1)       # 가장 큰 확률의 클래스 선택
  accuracy = (predicted == y_test).sum().item() / y_test.size(0)     # 일치 비율
  print(loss.item(), accuracy)       # 마지막 학습 손실, 테스트 정확도 출력

0.620134174823761 1.0


**순전파(Forward Propagation)**   
데이터를 입력에서 출력까지 한 방향으로 흘려보내며 계산하는 과정

각 층에서

1. 가중합: z = x·W + b (입력 × 가중치 + 편향)

2. 활성화: a = 활성화함수(z) (예: 시그모이드, ReLU)

이 값이 다음 층으로 넘어가며 또 같은 식으로 계산

-> 결국 마지막 층에서 나온 값(로짓 또는 softmax 확률)을 예측으로 씀


**역전파(Backpropagation)**   
예측이 정답과 얼마나 틀렸는지(손실) 를 계산하고 그게 어느 가중치 때문인지 한 층 한 층 거꾸로 따져서(미분/연쇄법칙) 가중치들을 고치는 과정

한 사이클(= 한 번의 학습 스텝)

1. 순전파: 예측을 만든다.

2. 손실(loss): 예측 vs 정답 차이(예: 교차엔트로피).

3. 기울기(gradient): 손실을 각 가중치로 미분(영향도).

4. 업데이트: W ← W - 학습률 * gradient (경사하강)

반복!



# keras 함수형 api로 신경망 구성하기

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import concatenate, Activation

input = Input(shape=(4,))
dense1 = Dense(50, activation='relu')(input)
dense2 = Dense(80, activation='relu')(input)
dense3 = Dense(30, activation='relu')(input)

x = concatenate([dense1, dense2, dense3])
output = Dense(3, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50)        │        250 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 80)        │        400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 30)        │        150 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 160)       │          0 │ dense_4[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        483 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,283 (5.01 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(X_train, y_train, epochs=300, verbose=True)
history

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2858 - loss: 2.8645
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2986 - loss: 2.1625
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5435 - loss: 1.6070
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5241 - loss: 1.2978 
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5119 - loss: 1.1621
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6080 - loss: 0.9481
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7787 - loss: 0.8603
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4766 - loss: 0.8535
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6261 - loss: 0.7956
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7846 - loss: 0.7642
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8637 - loss: 0.7303
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6892 - l

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
loss, acc

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0528 


(0.04787924140691757, 1.0)

## 예측

In [ ]:
import numpy as np
pred_proba = model.predict(X_test[:5])
pred_class = np.argmax(pred_proba, axis=1)
pred_class

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


array([1, 0, 2, 1, 1])

## 평가 결과

In [ ]:
from sklearn.metrics import classification_report
y_pred = pred_proba = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        19
         1.0       1.00      1.00      1.00        13
         2.0       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



# 모델의 저장

In [ ]:
model.save('iris_model.keras')

# 모델의 로드

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/iris_model.keras')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50)        │        250 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 80)        │        400 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 30)        │        150 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 160)       │          0 │ dense_4[0][0],    │
│ (Concatenate)       │                   │            │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        483 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,851 (15.05 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,568 (10.04 KB)

In [ ]:
json_str = model.to_json()
json_str

'{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional_1", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 4], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer_1"}, "registered_name": null, "name": "input_layer_1", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense_4", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "units": 50, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "kernel_constraint": null, "bias_constraint":

In [ ]:
import json
json_dic = json.loads(json_str)
with open('iris_model.json', 'w') as f:
  json.dump(json_dic, f, indent=4)    # 모델을 json 형식으로 바꾸기